In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
from pydrake.all import InverseKinematics

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba, VPolytope, BodyIndex,JacobianWrtVariable
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions


In [4]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope,
                                               SeparatingPlane,
                                               SeparatingPlaneOrder)
from pydrake.common import (
    FindResourceOrThrow, )
from pydrake.systems.framework import (Context, Diagram, DiagramBuilder)
from pydrake.multibody.plant import (AddMultibodyPlantSceneGraph,
                                     CoulombFriction, MultibodyPlant)
from pydrake.multibody.rational import RationalForwardKinematics
from pydrake.multibody.tree import (ModelInstanceIndex)
from pydrake.multibody.parsing import Parser
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.math import (RigidTransform, RollPitchYaw)
from pydrake.solvers.mosek import MosekSolver
from pydrake.solvers.osqp import OsqpSolver
from pydrake.solvers import mathematicalprogram as mp

In [5]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

In [6]:
from ur3e_demo import UrDiagram
import ur3e_demo
import visualization_utils as viz_utils
import networkx as nx

# Build and set up the visualization the plant



In [7]:
MosekSolver.AcquireLicense()
bilinear_alt_result_file = "dual_ur3e_bil_alt_result.npz"
load_data = bilinear_alt_result_file
weld_wrist = False
with_gripper = True

In [ ]:
ur3e_demo.search_dual_arm_cspace_polytope(
            weld_wrist=weld_wrist,
            with_gripper=with_gripper,
            binary_search_result_file=bilinear_alt_result_file)

INFO:drake:Meshcat listening for connections at http://localhost:7000


http://localhost:7000


INFO:drake:SOS 0/213 dispatched
INFO:drake:SOS 0/213 completed, is_success true
INFO:drake:SOS 1/213 dispatched
INFO:drake:SOS 1/213 completed, is_success true
INFO:drake:SOS 2/213 dispatched
INFO:drake:SOS 2/213 completed, is_success true
INFO:drake:SOS 3/213 dispatched
INFO:drake:SOS 3/213 completed, is_success true
INFO:drake:SOS 4/213 dispatched
INFO:drake:SOS 4/213 completed, is_success true
INFO:drake:SOS 5/213 dispatched
INFO:drake:SOS 5/213 completed, is_success true
INFO:drake:SOS 6/213 dispatched
INFO:drake:SOS 6/213 completed, is_success true
INFO:drake:SOS 7/213 dispatched
INFO:drake:SOS 7/213 completed, is_success true
INFO:drake:SOS 8/213 dispatched
INFO:drake:SOS 8/213 completed, is_success true
INFO:drake:SOS 9/213 dispatched
INFO:drake:SOS 9/213 completed, is_success true
INFO:drake:SOS 10/213 dispatched
INFO:drake:SOS 10/213 completed, is_success true
INFO:drake:SOS 11/213 dispatched
INFO:drake:SOS 11/213 completed, is_success true
INFO:drake:SOS 12/213 dispatched
INF

INFO:drake:SOS 100/213 completed, is_success true
INFO:drake:SOS 101/213 dispatched
INFO:drake:SOS 101/213 completed, is_success true
INFO:drake:SOS 102/213 dispatched
INFO:drake:SOS 102/213 completed, is_success true
INFO:drake:SOS 103/213 dispatched
INFO:drake:SOS 103/213 completed, is_success true
INFO:drake:SOS 104/213 dispatched
INFO:drake:SOS 104/213 completed, is_success true
INFO:drake:SOS 105/213 dispatched
INFO:drake:SOS 105/213 completed, is_success true
INFO:drake:SOS 106/213 dispatched
INFO:drake:SOS 106/213 completed, is_success true
INFO:drake:SOS 107/213 dispatched
INFO:drake:SOS 107/213 completed, is_success true
INFO:drake:SOS 108/213 dispatched
INFO:drake:SOS 108/213 completed, is_success true
INFO:drake:SOS 109/213 dispatched
INFO:drake:SOS 109/213 completed, is_success true
INFO:drake:SOS 110/213 dispatched
INFO:drake:SOS 110/213 completed, is_success true
INFO:drake:SOS 111/213 dispatched
INFO:drake:SOS 111/213 completed, is_success true
INFO:drake:SOS 112/213 dis

In [ ]:
load_data = np.load(load_file)
C = load_data["C"]
d = load_data["d"]
print(f"d: {d}")
ur_diagram = UrDiagram(num_ur=2,
                       weld_wrist=weld_wrist,
                       add_shelf=False,
                       add_gripper=with_gripper)
diagram_context = ur_diagram.diagram.CreateDefaultContext()
plant_context = ur_diagram.plant.GetMyMutableContextFromRoot(
    diagram_context)
scene_graph_context = ur_diagram.scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
rational_forward_kin = RationalForwardKinematics(ur_diagram.plant)
q_star = np.zeros((12, ))
ur_diagram.plant.SetPositions(plant_context, q_star)
ur_diagram.diagram.ForcedPublish(diagram_context)


In [ ]:
save_posture_file = "dual_ur_closest.npz"
ur3e_demo.sample_closest_posture(ur_diagram, rational_forward_kin, q_star,
                       plant_context, C, d, save_posture_file)
saved_posture_data = np.load(save_posture_file)

In [ ]:
inspector = ur_diagram.scene_graph.model_inspector()
# Set robot link color. Hightlight the body with the closest distance.
for i in range(2):
    for body_index in ur_diagram.plant.GetBodyIndices(
            ur_diagram.ur_instances[i]) + ur_diagram.plant.GetBodyIndices(
                ur_diagram.gripper_instances[i]):
        for body_geometry in inspector.GetGeometries(
                ur_diagram.plant.GetBodyFrameIdOrThrow(body_index)):
            if inspector.GetName(
                    body_geometry) != saved_posture_data["min_pair"][i]:
                rgba = np.array([0.5, 0.5, 0.5, 1])
            else:
                rgba = np.array([1., 0, 0, 1])
            SetDiffuse(ur_diagram.plant, ur_diagram.scene_graph,
                       body_index, inspector.GetName(body_geometry), rgba,
                       scene_graph_context)
ur_diagram.plant.SetPositions(plant_context, saved_posture_data["min_q"])
ur_diagram.diagram.ForcedPublish(diagram_context)
# pdb.set_trace()


In [ ]:
# Set the robot color
for i in range(2):
    for body_index in ur_diagram.plant.GetBodyIndices(
            ur_diagram.ur_instances[i]) + ur_diagram.plant.GetBodyIndices(
                ur_diagram.gripper_instances[i]):
        for body_geometry in inspector.GetGeometries(
                ur_diagram.plant.GetBodyFrameIdOrThrow(body_index)):
            SetDiffuse(ur_diagram.plant, ur_diagram.scene_graph,
                       body_index, inspector.GetName(body_geometry),
                       np.array([0.5, 0.5, 0.5, 1]), scene_graph_context)

ur_diagram.plant.SetPositions(plant_context, saved_posture_data["max_q"])
ur_diagram.diagram.ForcedPublish(diagram_context)